In [1]:
import pandas as pd

# url = "https://github.com/AquaticEcoDynamics/tassielakes-data/raw/main/data-lake/HT/Hydrology/462.1_WoodsLakeAtDam_SpillwayDischarge_Continuous.csv"

# Read the CSV file and select only the first column
df = pd.read_csv("../../data-lake/HT/Hydrology/462.1_WoodsLakeAtDam_SpillwayDischarge_Continuous.csv", usecols=[0], header=None, encoding='unicode_escape')
#df = pd.read_csv(url, usecols=[0], header=None, encoding='unicode_escape')

# Reset the index of the DataFrame
df = df.reset_index(drop=True)

# Name the first column
df.columns = ["Raw"]

# Split the 'Raw' column by multiple spaces
split_data = df["Raw"].str.split(r'\s+', expand=True)
#print(split_data)

# Split the 'Raw' column by multiple spaces
split_data = df["Raw"].str.split(r'\s+', expand=True)

# Extract Date and Time
df['Date'] = split_data[0] + ' ' + split_data[2]

# Extract Data value
df['Data'] = split_data[3]

# Drop the original 'Raw' column
df = df[['Date', 'Data']]

# Convert 'Data' to numeric
df['Data'] = pd.to_numeric(df['Data'], errors='coerce')
print(df)


                        Date  Data
0        09/01/1968 09:00:00   0.0
1        26/01/1968 09:00:00   0.0
2        05/02/1968 09:00:00   0.0
3        24/02/1968 09:00:00   0.0
4        30/03/1968 09:00:00   0.0
...                      ...   ...
1319984  16/04/2024 09:30:00   0.0
1319985  16/04/2024 09:45:00   0.0
1319986  16/04/2024 10:00:00   0.0
1319987  16/04/2024 10:15:00   0.0
1319988  16/04/2024 10:30:00   0.0

[1319989 rows x 2 columns]


In [24]:
# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

# Drop rows with NaT in 'Date' column
df = df.dropna(subset=['Date'])

print(df)

                       Date  Data
0       1968-01-09 09:00:00   0.0
1       1968-01-26 09:00:00   0.0
2       1968-02-05 09:00:00   0.0
3       1968-02-24 09:00:00   0.0
4       1968-03-30 09:00:00   0.0
...                     ...   ...
1319984 2024-04-16 09:30:00   0.0
1319985 2024-04-16 09:45:00   0.0
1319986 2024-04-16 10:00:00   0.0
1319987 2024-04-16 10:15:00   0.0
1319988 2024-04-16 10:30:00   0.0

[1319978 rows x 2 columns]


In [25]:
# Group by 'Date' and average the 'Data' values for duplicate timestamps
df = df.groupby('Date').mean().reset_index()

# Set 'Date' as index for resampling
df.set_index('Date', inplace=True)

# Resample the DataFrame to 15-minute intervals and interpolate
df = df.resample('15T').interpolate(method='linear')

# Reset the index to convert it back to a column
df = df.reset_index()

# Resample the DataFrame to hourly intervals and interpolate
# df = df.resample('H').interpolate(method='linear')

print(df)

                       Date  Data
0       1968-01-09 09:00:00   0.0
1       1968-01-09 09:15:00   0.0
2       1968-01-09 09:30:00   0.0
3       1968-01-09 09:45:00   0.0
4       1968-01-09 10:00:00   0.0
...                     ...   ...
1972994 2024-04-16 09:30:00   0.0
1972995 2024-04-16 09:45:00   0.0
1972996 2024-04-16 10:00:00   0.0
1972997 2024-04-16 10:15:00   0.0
1972998 2024-04-16 10:30:00   0.0

[1972999 rows x 2 columns]


In [26]:
# Write the DataFrame to a CSV file
df.to_csv("Interpolated_462.1_WoodsLakeAtDam_SpillwayDischarge_Continuous.csv", index=False)